# 第5章 誤差伝播法

In [1]:
# %cd /content/drive/MyDrive/work/
# !git clone https://github.com/oreilly-japan/deep-learning-from-scratch.git

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')
# %cd /content/drive/MyDrive/work/deep-learning-from-scratch/ch04/
# %cd /deep-learning-from-scratch

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import logging
import sys
import os
from pathlib import Path

# importディレクトリの追加
# sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
sys.path.append(os.path.join(Path().resolve(), 'refs'))
print(sys.path)


['/home/jovyan/work/dl_study/01_deep-learning-from-scratch', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/jovyan/.ipython', '/home/jovyan/work/dl_study/01_deep-learning-from-scratch/refs']


In [ ]:
import urllib.request
print(urllib.request.getproxies())

## 5.1 計算グラフ

T.B.W

## 5.2 連鎖率

T.B.W

## 5.3 逆伝播

T.B.W

## 5.4 単純なレイヤの実装

### 5.4.1 乗算レイヤの実装

- レイヤは`foward()`と`backward()`の共通メソッドを持つようにする
- 乗算レイヤはMulLayerという名前にする

In [5]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y  # 順伝播ではxとyを乗算するだけ
        
        return out
        
    def backward(self, dout):
        # パラメータXにおける偏微分(dL/dx)
        dx = dout * self.y    # 逆伝播時はxとyをひっくり返す
        # パラメータXにおける偏微分(dL/dy)        
        dy = dout * self.x
        
        return dx, dy

### 乗算レイヤの使用例

以下の図5-16の実装を行う

![fig5-16](./images/fig5-16.PNG)

In [6]:
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# foward(順伝播)
apple_price = mul_apple_layer.forward(apple, apple_num)    # 1層目
price = mul_tax_layer.forward(apple_price, tax)            # 2層目

print(f'price: {price}')

# backward(逆伝播)
dprice = 1    # 最終金額の逆伝播は定数なので1

dapple_price, dtax = mul_tax_layer.backward(dprice)          # 2層目
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # 1層目

print(f'dapple_price: {dapple_price}, dtax: {dtax}')
print(f'dapple: {dapple}, dapple_num: {dapple_num}')

price: 220.00000000000003
dapple_price: 1.1, dtax: 200
dapple: 2.2, dapple_num: 110.00000000000001


### 5.4.2. 加算レイヤの実装

- 加算レイヤは初期化が必要ない
- `foward()`では, 加算レイヤでは2つの引数`x, y`を受け取り、それを加算して出力する
- `backward()`では, 蒸留から伝わってきた微分(dout)をそのまま下流に流す

In [7]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy


### 加算レイヤと乗算レイヤの使用例

- 以下の図5-17を実装する

![fig5-17](./images/fig5-17.PNG)

In [8]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# foward
apple_price = mul_apple_layer.forward(apple, apple_num)               # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)           # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)                         # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)                         # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)           # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)               # (1)

print(f'price: {price}')  # -> 715
print(f'dapple_num: {price}, dapple: {dapple}')           # => 2.2, 110
print(f'dorange_num: {dorange_num}, dorange: {dorange}')  # => 3.3, 165
print(f'dtax: {dtax}')    # => 650 

price: 715.0000000000001
dapple_num: 715.0000000000001, dapple: 2.2
dorange_num: 165.0, dorange: 3.3000000000000003
dtax: 650


## 5.5 活性化関数レイヤの実装

### 5.5.1 ReLUレイヤ

ReLU(Rectified Linear Unit)は次の式で表せた (5.7)

$$
    ReLU(y) =
        \begin{cases}
            x \quad (x > 0) \\
            0 \quad (x \leqq 0) \\
        \end{cases}
$$

ReLU(5.7)のxに関するyの微分は以下のように求められる(5.8)

$$
    \frac{\partial y}{\partial x} =
        \begin{cases}
            1 \quad (x > 0) \\
            0 \quad (x \leqq 0) \\
        \end{cases}
$$

![fig5-18](./images/fig5-18.PNG)


- **順伝播時の入力xが0より大きければ, 逆伝播は上流の値をそのまま下流に流す \[左図\]** 

- **順伝播時の入力xが0以下のときは, 逆伝播では何も流さない(0を流す) \[右図\]** 

In [9]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

x = np.array( [[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


- mask変数はTrue/FalseからなるNumpy配列
- 順伝播の入力であるxの要素で0以下の場所をTrue, それ以外をFalseとして保持する
- 逆伝播では順伝播時に保持したmaskを使い, 上流からdoutにおける0以下の要素を0に設定する

## 5.5.2 Sigmoidレイヤ

- **「exp」ノード**は, $ y = \exp(x) $の計算を行う
- **「／」ノード**は, $ y = \frac{1}{x} $の計算を行う

![fig5-20](./images/fig5-20.PNG)

### Step1. 
- $ y = \frac{1}{x} $の微分は解析的に次の式で表される
$$
    \frac{\partial y}{\partial x} = -\frac{1}{x^2} = -y^2
$$

- 逆伝播のときは, 上流の値に対して$ -y^2 $を乗算して下流に伝播する
  - (順伝播の出力の2乗にマイナスをつけた値)
  
### Step2.
- "+"ノードは上流の値を下流にそのまま流すだけ

### Step3.

- "exp"ノードの微分は以下で表される

$$
    \frac{\partial y}{\partial x} = \exp(x)
$$
- sigmoid関数は微分しても, sigmoid関数となる

### Step4. 
- "×"ノードは, 順伝播時の値をひっくり返して乗算する

---

- 図5-20の計算グラフの逆伝播出力は $ \frac {\partial L}{\partial y} y^2 \exp(-x) $となる

- これはさらに整理して書くと以下のようになる

$$

    \frac{\partial L}{\partial y} y^2 \exp(-x) = \frac{\partial L}{\partial y}\frac{1}{(1 + \exp(-x)^2)^2}\exp(-x) \\    
    \frac{\partial L}{\partial y} y(1 - y)
$$

![fig5-22](./images/fig5-22.PNG)

- **sigmoidレイヤは順伝播時に出力を保持しておいて、逆伝播時の計算に利用するのがポイント**

In [10]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        
        # 順伝播時に出力をoutに保持しておく
        self.out = out
        
        return out

    def backward(self, dout):
        # 逆伝播時に過去の出力結果を用いる
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

## 5.6 Affine／Softmaxレイヤの実装

- nnの順伝播では重み付き信号の総和計算に **行列の積**`np.dot()`を利用した

- ここでXを入力, Wを重み, Bをバイアスとし、それぞれ(2,), (2,3), (3)の多次元配列であるとする

- そうするとNNの重み付き和は`Y = np.dot(X,W) + B`のように計算できる

- 行列の積の計算は対応する次元の要素数を一致させるのがポイント

- NNの順伝播で行う行列の積は幾何学分野ではアフィン変換と呼ばれる
    - アフィン変換を行う処理をAffineレイヤという名前で実装する

- 行列を対象とした逆伝播を求める場合, 行列の要素後に書き下すことでスカラを対象とした計算グラフと同じ手順で考えることができる

$$
    \frac{\partial L}{\partial X} = \frac{\partial L}{\partial \boldsymbol{Y}} \cdot W^T \\
    \frac{\partial L}{\partial W} = X^T \cdot \frac{\partial L}{\partial \boldsymbol{Y}}
$$

![fig5-25](./images/fig5-25.PNG)

### 5.6.2 バッチ版Affineレイヤ

- N個のデータをまとめて入力可能なAffineレイヤを考える

- 入力であるXの形状が(N, 2)になっただけ

- 前と同じように計算グラフ上で, 素直に行列の計算をするだけ

- 逆伝播の際は, 行列の形状に注意すれば(?), スカラの場合と同様に計算できる

- バイアスの加算は注意が必要

  - 順伝播のバイアス加算は X・Wに対して, バイアスがそれぞれ加算される
  
  - 逆伝播の際には, **それぞれのデータの逆伝播の値がバイアスの要素に集約される必要がある**(?)
  
  - バイアスの逆伝播はその2個のデータに対しての微分を, **データ毎に合算して求める**

In [11]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

print(X_dot_W)

print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [12]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
print(dY)

dB= np.sum(dY, axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [13]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.w) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dw = np.dot(self.x.T, dout)
        
        # np.sum()で0番目の軸に対しての総和を求め, バッチ単位で合算する
        self.db = np.sum(dout, axis=0)
        
        return dx

### 5.6.3 Softmax-with-Lossレイヤ

- Softmaxレイヤは, 入力された値を正規化(総和が1になるように変形)する

- 損失関数であるクロスエントロピー誤差も含めた"Softmax-with-Loss"レイヤを実装する

![fig5-28](./images/fig5-28.PNG)

![fig5-29](./images/fig5-29.PNG)

- ここでは、3 クラス分類を行う場合を想定し、前レイヤから3 つの入力（スコア）を受け取るものとする

- Softmaxレイヤは, 入力である$ (a_1, a_2, a_3) $を正規化して, $ (y_1, y_2, y_3) $を出力する

- Cross Entropy Errorレイヤは, Softmaxの出力$ (y_1, y_2, y_3) $と教師ラベル$ (t_1, t_2, t_3) $を受け取り, 損失$ L $を出力する

- Softmaxレイヤからの逆伝播は, $ (y_1-t_1, y_2-t_2 , y_-t_3 ) $という**Softmaxレイヤの出力と教師ラベルの差分になる**


## 具体例

- 教師ラベルが$ (0, 1, 0) $であるデータに対して、Softmaxレイヤの出力が$ (0.3, 0.2, 0.5) $ であった場合を考える。

  - 正解ラベルに対する確率は20%なので, この時点ではNNの学習が不足している
  
  - この場合, Softmaxレイヤからの逆伝播は$ (0.3, -0.8, 0.5) $という**大きな誤差が伝播される**
  
  - その結果, Softmaxよりも前のレイヤは, **<u>その大きな誤差から大きな内容を学習することになる</u>**

In [14]:
class SoftmaxWithLoss:
    def __init__(self):
        
        self.loss = None  # 損失
        self.y = None     # Softmaxの出力
        self.t = None     # 教師データ(one-hot表現)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
        
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

## 5.7 誤差逆伝播法の実装

- レイヤを組み合わせることで、レゴブロックを組み合わせるようにNNを構築することができる
- 数値微分は簡単に実装できる反面, 計算に時間がかかる
- 誤差逆伝播法は計算時間を早くでき, 効率よく勾配を求めることができる

### 5.7.1 NNの学習の全体図

- ニューラルネットワークは、適応可能な**重み**と**バイアス**があり、<u>この重みとバイアスを訓練データに適応するように調整することを「学習」と呼ぶ</u>
- ニューラルネットワークの学習は次の4 つの手順で行う。

1. **(ミニバッチ)** 訓練データの中からランダムに一部のデータを選び出す
2. **(勾配の算出)** 各重みパラメータに関する損失関数の勾配を求める
3. **(パラメータの更新)** 重みパラメータを勾配方向に微小量だけ更新する。
4. **(繰り返す)** ステップ1～3を繰り返す

In [15]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        
        self.params = {}
        # 重みは正規分布に基づく分散0.01の乱数で初期化する
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        # バイアスはゼロで初期化する
        self.params['b1'] = np.zeros(hidden_size)
        
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(hidden_size)
        
        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def loss(self, x, t):
        '''データとラベルを入力し、そのlossを返す
        args
          x: 入力データ
          t: 教師データ
        '''
        
        # 最終層(SoftmaxwithLoss)直前までの順伝播する
        y = self.predict(x)
        
        # 最終層に直前のレイヤの出力を入力し, Lossを返す
        return self.lastLayer.forward(y, t)
        
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        
        # 入力された教師データが複数ある場合は, 最も大きいものを返す(One-Hot表現対応?)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        # ラベルが一致する正解データの割合(精度)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        '''各層の偏微分を行い, 勾配を求める
        x: 入力データ
        t: 教師データ        
        '''
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_w, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_w, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_w, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        
        # backward
        dout = 1    # 最終層の微分結果は1(定数項しかないので)
        dout = self.lastLayer.backward(dout)
        
        # OrderedDictをreverseで逆順にすると, 後ろからの逆伝播を簡単に記述できる
        layers = list(self.layers.values())
        layers.reverse()        
        for layer in layers:
            dout = layer.backward(dout)
        
        # 設定
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

## 5.7.3 誤差逆伝播法の勾配確認

- 数値微分は<u>実装が簡単であるため、ミスが起きにくい</u>
- 誤差逆伝播法は<u>実装が複雑になるため, ミスが起きやすい</u>

- 数値微分の結果と誤差逆伝播法の結果を比較し, 誤差逆伝播法の実装の正しさを確認することがよく行われる**(勾配確認)**

In [16]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist
from ch05.two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 各重みの絶対誤差の平均を求める
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(f'{key}: {str(diff)}')

W1: 3.7143381633662783e-10
b1: 2.2330886705462104e-09
W2: 6.112464970068129e-09
b2: 1.4005502981112584e-07


## 5.7.4 誤差逆伝播法を使った学習

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from ch05.two_layer_net import TwoLayerNet

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]

batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
        
    # 誤差逆伝播法によって勾配を求める
    grad = network.gradient(x_batch, t_batch)
    
    # 重みとバイアスを更新する
    for key in ('W1', 'b1', 'W2', 'b2'):
        # 算出した勾配に LRを乗算し, 次の重みパラメータに設定する
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f'it[{i:4d}], train_acc: {train_acc}, test_acc: {test_acc}')

## 5.8 まとめ

- 計算グラフを用いてNNの誤差逆伝播法を説明
- ReLuレイヤやSoftmax-with-Lossレイヤ, Affineレイヤ, Softmaxレイヤ等を学んだ

- forward(), backwardというメソッドを実装することで

### 本章で学んだこと

- 計算グラフを用いれば, 計算過程を視覚的に把握することができる
- 計算グラフのノードは局所的な計算によって構成される. 局所的な計算が全体の計算を構成する
- 計算グラフの順伝播は, 通常の計算を行う. 
- 計算グラフの逆伝播では, 各ノードの微分を求まる
- NNの構成要素をレイヤとして実装し, 勾配計算を効率的に行える(**誤差逆伝播法**)
- 数値微分と誤差逆伝播法の結果を比較すると, 誤差逆伝播法の実装に誤りがないことを確認できる(**勾配確認**)